In [4]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Built-in Tool - DuckDuckGo Search


In [6]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [7]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('Asia Cup 2025 news')
print(results)

8 hours ago - However, following the Pahalgam ... meeting in Dhaka, ACC chairman Mohsin Naqvi announced that the tournament would instead be held in the United Arab Emirates, with matches staged in Dubai and Abu Dhabi .... The tournament will run throughout September 2025, with the final scheduled for 28th September . A total of 19 matches will be played — promising fans a month-long celebration of thrilling T20 cricket, perfectly timed as a build-up to the T20 World Cup 2026. 2 days ago - 25 September 2025 · Asia Cup, ... 3-17 · Pakistan won by 11 runs · Scorecard. Table · Pakistan set up an Asia Cup final showdown with India after successfully defending 136 to beat Bangladesh by 11 runs .... 2 hours ago - Holders India meet Pakistan for the first time in the final of an Asia Cup tournament with tensions running high. 3 weeks ago - Asia Cup 2025 : 'I’ll go on strike if Jasprit Bumrah plays': Ex-India captain que... How much of the $5 million US Open prize money will Carlos Alcaraz tak

In [17]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


Built-in Tool - Shell Tool

In [9]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.7 MB/s eta 0:00:00


In [10]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()

results = shell_tool.invoke('ls')
print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

Method 1: using `@tool` decorator

In [11]:
from langchain_core.tools import tool

In [12]:
# Step 1 - create a function
def multiply(a, b):
  """Multiply 2 numbers"""
  return a*b

In [13]:
# Step 2 - add type hints
def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a*b

In [14]:
# Step 3 - add tool decorator
@tool
def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a*b

In [15]:
result = multiply.invoke({"a": 2, "b": 3})
print(result)

6


In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply 2 numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [18]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply 2 numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


Method 2 - Using Structured tool and pydantic to create custom tool

A StructuredTool in LangChain is a special type of tool where the input to the tool follows a structured schema, typically defined using a Pydantic model

In [20]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [22]:
class MultiplyInput(BaseModel):
  a: int = Field(requied=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

In [23]:
def multiply_func(a: int, b: int) -> int:
  return a * b

In [24]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply 2 numbers",
    args_schema=MultiplyInput
)

In [25]:
result = multiply_tool.invoke({"a": 2, "b": 3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

6
multiply
Multiply 2 numbers
{'a': {'description': 'The first number to add', 'requied': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


Method 3 - Using BaseTool class

BaseTool is the abstract base class for all tools in LangChain. It defines the core structure and interface that any tool must follow, whether it's a simple one-liner or a fully customized function

All other tool types like @tool, StructuredTool are built on top of BaseTool

In [26]:
from langchain.tools import BaseTool
from typing import Type

In [27]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
  a: int = Field(requied=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

In [32]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Type

# arg schema using pydantic
class MultiplyInput(BaseModel):
  a: int = Field(requied=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

class MultiplyTool(BaseTool):
  name: str = "multiply"
  description: str = "Multiply two numbers"
  args_schema: Type[MultiplyInput] = MultiplyInput

  def _run(self, a: int, b: int) -> int:
    return a * b

In [33]:
multiply_tool_instance = MultiplyTool()
result = multiply_tool_instance.invoke({"a": 5, "b": 7})
print(result)

35


### Toolkit

In [34]:
# Custom tools
@tool
def add(a: int, b: int) -> int:
  """Add 2 numbers"""
  return a + b

@tool
def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a * b

In [35]:
class MathToolKit:
  def get_tools(self):
    return [add, multiply]

In [36]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name, '->', tool.description)

add -> Add 2 numbers
multiply -> Multiply 2 numbers
